In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
from random import randrange
import math

In [2]:
dataset = pd.read_csv('D:/RnD/Dynamic Pricing/Input Data/Ads_Optimisation.csv')

In [3]:
dataset.head()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
0,1,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0


In [44]:
dataset.iloc[0,0]

1

In [4]:
dataset.describe()

,Ad 1,Ad 2,Ad 3,Ad 4,Ad 5,Ad 6,Ad 7,Ad 8,Ad 9,Ad 10
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,0.170300,0.129500,0.072800,0.119600,0.269500,0.012600,0.111200,0.209100,0.095200,0.04890
std,0.375915,0.335769,0.259821,0.324509,0.443722,0.111546,0.314395,0.406686,0.293506,0.21567
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [5]:
dataset.sum(axis = 0, skipna = True)

Ad 1     1703
Ad 2     1295
Ad 3      728
Ad 4     1196
Ad 5     2695
Ad 6      126
Ad 7     1112
Ad 8     2091
Ad 9      952
Ad 10     489
dtype: int64

In [3]:
def random_reward():
    N = 10000
    d = 10
    ads_selected = []
    total_reward = 0
    for n in range(0, N):
        ad = random.randrange(d)
        ads_selected.append(ad)
        reward = dataset.values[n, ad]
        total_reward = total_reward + reward
    return total_reward

In [7]:
pd.Series(ads_selected).tail(1000).value_counts(normalize = True)

7    0.107
3    0.107
1    0.105
9    0.103
6    0.103
2    0.099
8    0.098
4    0.098
0    0.097
5    0.083
dtype: float64

In [4]:
def ucb_reward():
    N = 10000
    d = 10
    ads_selected = []
    numbers_of_selections = [0] * d
    sums_of_reward = [0] * d
    total_reward = 0

    for n in range(0, N):
        ad = 0
        max_upper_bound = 0
        for i in range(0, d):
            if (numbers_of_selections[i] > 0):
                average_reward = sums_of_reward[i] / numbers_of_selections[i]
                delta_i = math.sqrt(2 * math.log(n+1) / numbers_of_selections[i])
                upper_bound = average_reward + delta_i
            else:
                upper_bound = 1e400
            if upper_bound > max_upper_bound:
                max_upper_bound = upper_bound
                ad = i
        ads_selected.append(ad)
        numbers_of_selections[ad] += 1
        reward = dataset.values[n, ad]
        sums_of_reward[ad] += reward
        total_reward += reward
    return total_reward

In [9]:
total_reward

2125

In [10]:
len(dataset)

10000

In [16]:
sample_size = int(len(dataset)*0.05)

In [48]:
sample_size = int(len(dataset)*0.05)
sample_dataset = dataset.iloc[0:sample_size, :]
result_df = pd.DataFrame(0, index = np.arange(sample_size), columns = sample_dataset.columns)
total_reward = 0 
for i in range(len(sample_dataset)):
    j = randrange(10)
    result_df.iloc[i,j] = sample_dataset.iloc[i,j]
    #total_reward = total_reward + sample_dataset.iloc[i, j]

temp_x = result_df.sum(axis = 0, skipna = True)

In [50]:
temp_x

Ad 1     10
Ad 2      8
Ad 3      7
Ad 4      8
Ad 5     12
Ad 6      0
Ad 7      6
Ad 8     14
Ad 9      4
Ad 10     3
dtype: int64

In [5]:
def greedy_reward():
    sample_size = int(len(dataset)*0.05)
    sample_dataset = dataset.iloc[0:sample_size, :]

    reward_points = 0
    iterations = 100
    for n in range (iterations):
        result_df = pd.DataFrame(0, index = np.arange(sample_size), columns = sample_dataset.columns)
        total_reward = 0 
        #print("total start = ", total_reward)
        for i in range(len(sample_dataset)):
            j = randrange(10)
            result_df.iloc[i,j] = sample_dataset.iloc[i,j]
            #total_reward = total_reward + sample_dataset.iloc[i, j]
        total_reward = result_df.values.sum()
        #print(total_reward)
        temp_x = result_df.sum(axis = 0, skipna = True)
        ad_exploration = temp_x.idxmax()
        reset_dataset = dataset.iloc[sample_size:,:]

        total_reward = total_reward + reset_dataset[ad_exploration].sum()
        reward_points = reward_points + total_reward
        #print(len(reset_dataset))
    #print(ad_exploration)
    total_reward = reward_points/iterations 
    return total_reward

In [13]:
 while True: 
    print("Hello! Please enter 1 to continue & 0 to exit ") 
    input_selection = int(input()) 
    if (input_selection !=1): 
        print("Quiting now!") 
        break 
    print("Hello! Please enter which MAB algorithms you might want to select:") 
    #adstock = float(input()) 
    print ("Enter 1 for MAB using Random Selection") 
    print ("Enter 2 for MAB using Upper Confidence Bound Methodology") 
    print ("Enter 3 for MAB using Upper Greedy Methodology") 
    print ("Enter 4 for MAB using Epsilon Greedy Methodology") 
    print ("Enter 5 for MAB using Stepwise Explore & Exploit Methodology") 
    print ("Enter 6 for MAB using Thompson Sampling Methodology")
    model_choice = int(input()) 
    if (model_choice == 1): 
        print ("Your model selection is Random Selection") 
        reward = random_reward() 
    elif (model_choice == 2): 
        print ("Your model selection is UCB Methodology") 
        reward = ucb_reward() 
    elif (model_choice == 3): 
        print ("Your model selection is Greedy Methodology") 
        reward = greedy_reward() 
    elif (model_choice == 4): 
        print ("Your model selection is Epsilon Greedy Methodology") 
        reward = epsilon_greedy_reward() 
    elif (model_choice == 5): 
        print ("Your model selection is Stepwise Explore & Exploit Methodology") 
        reward = stepwise_explore_exploit() 
    elif (model_choice == 6): 
        print ("Your model selection is Thompson Sampling Methodology") 
        reward = thompson_sampling()
    else: 
        print("Your selection is wrong, now quiting!") 
        break 
    print("Your reward on the basis of your model selection is:", reward)

Hello! Please enter 1 to continue & 0 to exit 
1
Hello! Please enter which MAB algorithms you might want to select:
Enter 1 for MAB using Random Selection
Enter 2 for MAB using Upper Confidence Bound Methodology
Enter 3 for MAB using Upper Greedy Methodology
Enter 4 for MAB using Epsilon Greedy Methodology
Enter 5 for MAB using Stepwise Explore & Exploit Methodology
Enter 6 for MAB using Thompson Sampling Methodology
1
Your model selection is Random Selection
Your reward on the basis of your model selection is: 1238
Hello! Please enter 1 to continue & 0 to exit 
1
Hello! Please enter which MAB algorithms you might want to select:
Enter 1 for MAB using Random Selection
Enter 2 for MAB using Upper Confidence Bound Methodology
Enter 3 for MAB using Upper Greedy Methodology
Enter 4 for MAB using Epsilon Greedy Methodology
Enter 5 for MAB using Stepwise Explore & Exploit Methodology
Enter 6 for MAB using Thompson Sampling Methodology
2
Your model selection is UCB Methodology
Your reward on 

In [26]:
temp_x = dataset.iloc[0]
dataset[temp_x.idxmax()].iloc[0]
#print(temp_x)

1

In [6]:
def func_explore(i):
    temp_x = dataset.iloc[i]
    return temp_x.idxmax()

In [7]:
def func_exploit(ad, i):
    reward = dataset[ad].iloc[i]
    return reward

In [8]:
def epsilon_greedy_reward():
    reward_list = []
    ad = func_explore(0)
    for i in range(len(dataset)):
        p = random.randrange(100)
        if p < 10:
            ad = func_explore(i)
            reward = func_exploit(ad, i)
        else: 
            reward = func_exploit(ad, i)
        reward_list.append(reward)
        total_reward = sum(reward_list)
    return total_reward
#print(random.randrange(100))

# Greedy Epsilon Model
p = random number
if p < e:            ## e: epsilon being a small number
    Explore
Else: 
    Exploit

# Stepwise Explore Exploit

Loop: 
    5% Explore
    20% Exploit

In [39]:
len(dataset.columns)

10

In [9]:
def pc_5_explore(sample_dataset):
    
    df_len = len(sample_dataset)
    rand = len(sample_dataset.columns)
    result_df = pd.DataFrame(0, index = np.arange(df_len), columns = sample_dataset.columns)
    total_reward = 0 
        #print("total start = ", total_reward)
    for i in range(len(sample_dataset)):
        j = randrange(rand)
        result_df.iloc[i,j] = sample_dataset.iloc[i,j]
        #total_reward = total_reward + sample_dataset.iloc[i, j]
    total_reward = result_df.values.sum()
    #print(total_reward)
    temp_x = result_df.sum(axis = 0, skipna = True)
    ad_exploration = temp_x.idxmax()
    return total_reward, ad_exploration

In [10]:
def pc_20_exploit(sample_dataset, ad): 
    reward = 0
    reward = reward + sample_dataset[ad].sum()
    return reward
    

In [11]:
def stepwise_explore_exploit(): 
    total_reward = 0
    n = 10
    for itr in range(n):
        pc_5_val = int(len(dataset)*0.05)
        pc_20_val = int(len(dataset)*0.2)

        sample_dataset = dataset.iloc[0:pc_5_val, :]

        reward, ad = pc_5_explore(sample_dataset)
        total_reward = total_reward + reward

        sample_dataset = dataset.iloc[pc_5_val:pc_20_val, :]

        reward = pc_20_exploit(sample_dataset, ad)
        total_reward = total_reward + reward

        new_limit = pc_5_val + pc_20_val
        sample_dataset = dataset.iloc[pc_20_val:new_limit, :]

        reward, ad = pc_5_explore(sample_dataset)
        total_reward = total_reward + reward

        sample_dataset = dataset.iloc[new_limit:, :]

        reward = pc_20_exploit(sample_dataset, ad)
        total_reward = total_reward + reward
    return total_reward/n

In [12]:
def thompson_sampling():
    observations = len(dataset)
    machines = len(dataset.columns)
    machine_selected = []
    rewards = [0] * machines
    penalties = [0] * machines
    total_reward = 0
    for n in range(0, observations):
        bandit = 0
        beta_max = 0
        for i in range(0, machines):
            beta_d = random.betavariate(rewards[i] + 1, penalties[i] + 1)
            if beta_d > beta_max:
                beta_max = beta_d
                bandit = i
        machine_selected.append(bandit)
        #print(bandit)
        reward = dataset.values[n, bandit]
        if reward == 1:
            rewards[bandit] = rewards[bandit] + 1
        else:
            penalties[bandit] = penalties[bandit] + 1
        #print(reward)
        total_reward = total_reward + reward
    return total_reward